In [9]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

Base = declarative_base()

class Student(Base):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    subjects = relationship('Subject', secondary='student_subject', back_populates='students')

class Subject(Base):
    __tablename__ = 'subjects'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    students = relationship('Student', secondary='student_subject', back_populates='subjects')

class Student_Subject(Base):
    __tablename__ = 'student_subject'
    student_id = Column(Integer, ForeignKey('students.id'), primary_key=True)
    subject_id = Column(Integer, ForeignKey('subjects.id'), primary_key=True)

    student = relationship('Student', backref='student_subject')
    subject = relationship('Subject', backref='student_subject')

engine = create_engine('sqlite:///school.db')
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

student1 = Student(name='Art')
student2 = Student(name='Karl')
student3 = Student(name='Shtrih')

subject1 = Subject(name='Math')
subject2 = Subject(name='Science')
subject3 = Subject(name='English')

session.add_all([student1, student2, student3, subject1, subject2, subject3])
session.commit()

student_subject1 = Student_Subject(student=student1, subject=subject1)
student_subject2 = Student_Subject(student=student2, subject=subject3)
student_subject3 = Student_Subject(student=student3, subject=subject2)

session.add_all([student_subject1, student_subject2, student_subject3])
session.commit()


C:\Users\Artem.Khavroniuk\AppData\Local\Temp\ipykernel_17644\536206171.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
C:\Users\Artem.Khavroniuk\AppData\Local\Temp\ipykernel_17644\536206171.py:33: SAWarning: relationship 'Student.student_subject' will copy column students.id to column student_subject.student_id, which conflicts with relationship(s): 'Student.subjects' (copies students.id to student_subject.student_id), 'Subject.students' (copies students.id to student_subject.student_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that

In [10]:
students_in_english = session.query(Student).join(Student_Subject).join(Subject).filter(Subject.name == 'English').all()
students_names_in_english = [student.name for student in students_in_english]
print(students_names_in_english)


['Karl', 'Karl']
